In [1]:
import itertools
from parlay_system import *
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)


In [19]:
rangers = MoneyLine(event="rangers", bet_amount=100, odds=160)
flyers = MoneyLine(event="flyers", bet_amount=100, odds=-190)

cbj = MoneyLine(event="cbj", bet_amount=100, odds=148)
islanders = MoneyLine(event="islanders", bet_amount=100, odds=-175)

hawaii = MoneyLine(event="hawaii", bet_amount=100, odds=230)
wash = MoneyLine(event="wash", bet_amount=100, odds=-295)

edm = MoneyLine(event="edm", bet_amount=100, odds=118)
van = MoneyLine(event="van", bet_amount=100, odds=-137)

binaries = [
    [rangers, flyers],
    [cbj, islanders],
    [hawaii, wash],
#     [edm, van]
]

results = [
    [0, 1], #[rangers, flyers],
    [0, 1], #[cbj, islanders],
    [0, 1], #[wash, hawaii],
#     [0, 1], #[edm, van],
]


override_arr = [0, 2, 99]


def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(binaries, results)

In [20]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [21]:
all_subsets

[(0, 1, 2)]

In [5]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.05,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup,
                    override_arr=override_arr
                    )
    
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.024078970274573994
     jac: array([-1.899182  , -1.30062993, -0.17635017,  0.06651379, -0.83703929,
       -0.45777242,  0.25461683,  0.40850528, -0.70194895, -0.35057222,
        0.30943006,  0.45200202, -0.07842387,  0.14422267,  0.56242688,
        0.65276626])
 message: 'Positive directional derivative for linesearch'
    nfev: 100
     nit: 9
    njev: 5
  status: 8
 success: False
       x: array([0.01      , 0.01      , 0.01303224, 0.01507409, 0.01      ,
       0.01051637, 0.01431567, 0.01333645, 0.01      , 0.0113511 ,
       0.01401692, 0.01295998, 0.01421564, 0.01480118, 0.01166804,
       0.01      ])
slsqp_solver: 
                           event       index[]        result  ev_stat         odds    bet       mult  payout  profit
15     flyers_islanders_wash_van  [1, 3, 5, 7]  [1, 1, 1, 1]     True   455.573984  0.010   5.555740  0.0556  -0.140
0         rangers_cbj_hawaii_edm  [0, 2, 4, 6]  [0, 0, 0, 0]    False  4538.691200  0.010  46.386912  0.4639   0.269

In [6]:
t = MoneyLine(event="neg", bet_amount=0.04600910801209846, odds=226.022404)
t.print_stats()

x = 0.15 / t.multiplier
print(x)

  event  bet_amount        odds  multiplier  payout
0   neg    0.046009  226.022404    3.260224    0.15
0.04600910801209846


In [7]:
t = MoneyLine(event="neg", bet_amount=0.02795640694842181, odds=436.549637)
t.print_stats()

x = 0.15 / t.multiplier
print(x)

  event  bet_amount        odds  multiplier  payout
0   neg    0.027956  436.549637    5.365496    0.15
0.02795640694842181


In [8]:
t = MoneyLine(event="neg", bet_amount=0.029153236661330132, odds=414.522630)
t.print_stats()

x = 0.15 / t.multiplier
print(x)

  event  bet_amount       odds  multiplier  payout
0   neg    0.029153  414.52263    5.145226    0.15
0.029153236661330132


In [9]:
df.describe()

,odds,bet,mult,payout,profit
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,1795.143504,0.012188,18.951435,0.219369,0.024063
std,1140.129906,0.001905,11.401299,0.105020,0.105202
min,455.573984,0.010000,5.555740,0.055600,-0.140000
25%,965.281811,0.010000,10.652818,0.147725,-0.047250
50%,1509.528062,0.012500,16.095281,0.235200,0.040000
75%,2330.106482,0.014000,24.301065,0.252050,0.056750
max,4538.691200,0.015000,46.386912,0.463900,0.269000


In [10]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['ev_stat', 'profit'], ascending=[False, True])
negative_df.describe()

,odds,bet,mult,payout,profit
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,796.067375,0.012667,8.960674,0.117117,-0.078333
std,240.329595,0.001506,2.403296,0.043433,0.043565
min,455.573984,0.010000,5.555740,0.055600,-0.140000
25%,644.286931,0.012250,7.442869,0.089675,-0.106000
50%,811.594152,0.013000,9.115942,0.119900,-0.075500
75%,965.281811,0.013750,10.652818,0.147725,-0.047250
max,1092.613075,0.014000,11.926131,0.170700,-0.025000


In [11]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['ev_stat', 'profit'], ascending=[False, False])
positive_df.describe()

,odds,bet,mult,payout,profit
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,2394.589181,0.011900,24.945892,0.280720,0.085500
std,1034.267407,0.002132,10.342674,0.078623,0.078787
min,1269.243730,0.010000,13.692437,0.202700,0.007000
25%,1664.648714,0.010000,17.646487,0.245300,0.050000
50%,2146.675715,0.011000,22.466757,0.245300,0.050000
75%,2785.226722,0.013750,28.852267,0.288500,0.093500
max,4538.691200,0.015000,46.386912,0.463900,0.269000


In [12]:
negative_df.head(200)

,event,index[],result,ev_stat,odds,bet,mult,payout,profit
15,flyers_islanders_wash_van,"[1, 3, 5, 7]","[1, 1, 1, 1]",True,455.573984,0.010,5.555740,0.0556,-0.140
14,flyers_islanders_wash_edm,"[1, 3, 5, 6]","[1, 1, 1, 0]",False,600.116987,0.012,7.001170,0.0817,-0.114
11,flyers_cbj_wash_van,"[1, 2, 5, 7]","[1, 0, 1, 1]",False,776.796760,0.013,8.767968,0.1136,-0.082
7,rangers_islanders_wash_van,"[0, 3, 5, 7]","[0, 1, 1, 1]",False,846.391545,0.013,9.463915,0.1262,-0.069
10,flyers_cbj_wash_edm,"[1, 2, 5, 6]","[1, 0, 1, 0]",False,1004.911900,0.014,11.049119,0.1549,-0.040
6,rangers_islanders_wash_edm,"[0, 3, 5, 6]","[0, 1, 1, 0]",False,1092.613075,0.014,11.926131,0.1707,-0.025


In [13]:
positive_df.head(200)

,event,index[],result,ev_stat,odds,bet,mult,payout,profit
0,rangers_cbj_hawaii_edm,"[0, 2, 4, 6]","[0, 0, 0, 0]",False,4538.691200,0.010,46.386912,0.4639,0.269
1,rangers_cbj_hawaii_van,"[0, 2, 4, 7]","[0, 0, 0, 1]",False,3581.007883,0.010,36.810079,0.3681,0.173
4,rangers_islanders_hawaii_edm,"[0, 3, 4, 6]","[0, 1, 0, 0]",False,2839.262857,0.010,29.392629,0.2939,0.099
8,flyers_cbj_hawaii_edm,"[1, 2, 4, 6]","[1, 0, 0, 0]",False,2623.118316,0.010,27.231183,0.2723,0.077
2,rangers_cbj_wash_edm,"[0, 2, 5, 6]","[0, 0, 1, 0]",False,1782.160271,0.013,18.821603,0.2453,0.050
5,rangers_islanders_hawaii_van,"[0, 3, 4, 7]","[0, 1, 0, 1]",False,2232.435871,0.011,23.324359,0.2453,0.050
9,flyers_cbj_hawaii_van,"[1, 2, 4, 7]","[1, 0, 0, 1]",False,2060.915559,0.011,21.609156,0.2453,0.050
12,flyers_islanders_hawaii_edm,"[1, 3, 4, 6]","[1, 1, 0, 0]",False,1625.478195,0.014,17.254782,0.2453,0.050
3,rangers_cbj_wash_van,"[0, 2, 5, 7]","[0, 0, 1, 1]",False,1393.577929,0.015,14.935779,0.2251,0.030
13,flyers_islanders_hawaii_van,"[1, 3, 4, 7]","[1, 1, 0, 1]",False,1269.243730,0.015,13.692437,0.2027,0.007


In [14]:
positive_df.groupby('event_status').count()

KeyError: 'event_status'

In [ ]:
negative_df.groupby('event_status').count()

In [ ]:
neg = negative_df.reset_index(drop=True).style.applymap(color_negative_red)
neg

In [ ]:
pos = positive_df.reset_index(drop=True).style.applymap(color_positive_green)
pos

In [ ]:
sf49ers = MoneyLine(event="parlay", bet_amount=100, odds=1002)
sf49ers.print_stats()

In [ ]:
pos = MoneyLine(event="pos", bet_amount=100, odds=-1020)
pos.print_stats()

In [ ]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-305)
neg.print_stats()

In [ ]:
neg = MoneyLine(event="neg", bet_amount=1.0, odds=-285)
neg.print_stats()